### Initialize

Your first peek to MEG data with Python.
Let's first set up the necessary modules - you know this already!
The last line is for defining plotting behavior.


In [4]:
import mne
import numpy as np
import sys
%matplotlib notebook
#import matplotlib.pyplot as plt

### Load the MEG raw data


We use here some raw data from the "multimodal" public data set. This is available via the MNE-Python module and pre-loaded here, so no need to import your own.

First, we need to tell the path of the data file, part of which is nicely available programmatically and part of which we choose. Path components can be combined in a handy, pythonic way to the final _filename_.
Then, we will use the FIF-file type specific function to read the raw data to a variable called _raw_. We will later check how this is organized.

In [ ]:
filename = mne.datasets.multimodal.get_path() + "/MEG/sample/sample_audvis_raw.fif"
raw = mne.io.read_raw_fif(filename, allow_maxshield=False, preload=True, verbose=True)

### Check data info

One important feature in MNE-Python is the data info. It is a struct that contains important information (metadata) about that instance of MEG data. What is a struct? It is a Python datatype, where the data object has _keys_ that can be referred to by _names_. Each keyword corresponds to a value, which can be of any data type. Let's see how this works in practice.

In [ ]:
print(raw.info)
print(raw.info["ch_names"][0])
print(raw.info["first_samp"])

### Plot raw data

We will next plot these raw data. Plotting the whole data here is a bit cumbersome, so we will make a 50-second copy of the data. At the same time, we will filter it to < 80 Hz frequency band. Study the below commands how these things are done. You can use the wonderful MNE-Python [API documentation](https://mne.tools/1.6/generated/mne.io.Raw.html) to search for help. Play a bit with the options in the following sections to see how changes affect the outcome! There is a context menu in the plot window, check the things you can do there, too.

In [ ]:
raw_50s = raw.copy().crop(100,150).filter(h_freq=80, l_freq=None, filter_length='auto', fir_design='firwin', phase='zero')
raw.plot(duration=10, butterfly=False, group_by='selection')

### Plot spectra

The spectrum of a signal shows it's frequency content. Usually, the brain activity measured with MEG is investigated in the range 1-100 Hz. We will now compute and display the mean spectrum of all magnetometer channels. We expect to see at least the alpha-band peak at around 10 Hz.

In [ ]:
#raw.plot_psd(tmax=np.inf, average=False);
raw_spectrum = raw.copy().pick_types(meg='mag')
raw_spectrum.plot_psd_topo(fmin=1, fmax=100, proj=True, n_fft=4096,
                           layout=mne.channels.read_layout('Vectorview-mag.lout'))

### Find experimental events

In [ ]:
events=mne.find_events(raw)
mne.viz.plot_events(events, sfreq=raw.info['sfreq'], color=None);

In [ ]:
epochs=mne.Epochs(raw.filter(l_freq=0.5,h_freq=40), events, tmin=-0.2, tmax=0.5, baseline=(None,0))
evokeds={str(id): epochs[str(id)].average() for id in set(events[:,2])}
mne.viz.plot_compare_evokeds(evokeds, picks=mne.pick_types(raw.info,meg='grad'));